In [1]:
import glob 
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import random
import time
import threading
import queue
import numpy as np

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()

In [3]:
# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

488

In [4]:
for actor in world.get_actors():
    if actor.type_id[:7] == "vehicle":
        ego_id = actor.id

In [5]:
ego_vehicle = world.get_actor(actor_id=ego_id)

In [6]:
ego_vehicle

In [ ]:
vehicles = []

In [7]:
bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
x = random.choice(world.get_map().get_spawn_points())

In [8]:
second_vehicle = world.try_spawn_actor(bp, x)

In [9]:
second_vehicle.set_autopilot(True)

In [10]:
ego_vehicle.get_location().distance(second_vehicle.get_location())

129.25477600097656

In [ ]:
for actor in world.get_actors().filter("traffic.traffic_light"):
    actor.set_state(carla.TrafficLightState.Green)
    actor.freeze

In [11]:
from agents.navigation.basic_agent import BasicAgent

agent = BasicAgent(ego_vehicle)

In [12]:
agent.set_destination(second_vehicle.get_location())

In [18]:
while True:
    agent.set_destination(second_vehicle.get_location())
    agent.ignore_traffic_lights(True)
    ego_vehicle.apply_control(agent.run_step())
    time.sleep(1)

KeyboardInterrupt: 

In [ ]:
def process_data(queue_data, output_folder):
    while True:
        try:
            # Wait for 1 second for data to be available in the queue
            data = queue_data.get(timeout=1.0)  

            # Create a filename using the timestamp of the data
            timestamp = str(int(time.time() * 1000))  # Milliseconds since epoch
            filename = os.path.join(output_folder, f'data_{timestamp}.txt')

            # Write the data to the file
            with open(filename, 'w') as f:
                f.write(str(data))

        except queue.Empty:
            continue

In [ ]:
output_folder = '/home/parsa/Cheetah/carla_simulator/out'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()

In [ ]:
# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

In [ ]:
# Set up the data processing thread
data_queue = queue.Queue()
data_thread = threading.Thread(target=process_data, args=(data_queue, output_folder))
data_thread.start()

In [ ]:
record_time = 60 # Seconds
time0 = time.time()
info = []
info2 = []
while True:
    # Synchronize the simulation
    world.tick()

    # Get data from the sensors
    info.append(vehicle.get_location())
    info.append(vehicle.get_velocity())
    info.append(vehicle.get_acceleration())

    info2.append(vehicle2.get_location())
    info2.append(vehicle2.get_velocity())
    info2.append(vehicle2.get_acceleration())
    
    # # Put the data into the queue
    # data = {
    #     'vehicle_location': (vehicle_location.x,vehicle_location.y,vehicle_location.z),
    #     'vehicle_velocity': (vehicle_velocity.x,vehicle_velocity.y,vehicle_velocity.z),
    #     'vehicle_acceleration': (vehicle_acceleration.x,vehicle_acceleration.y,vehicle_acceleration.z),
    # }
    # data_queue.put(data)

    time1 = time.time()
    if (time1-time0) > record_time:
        break
    
time.sleep(5)